<a href="https://colab.research.google.com/github/abhinayareddy1930/RNN-DL/blob/main/GPT2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets

In [8]:
# -*- coding: utf-8 -*-
"""DL_ASSIGNMENT_2_QUESTION2.ipynb

Automatically generated by Colab.
"""

# Install required libraries
!pip install transformers datasets --quiet

import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases

# Standard imports
import torch
from google.colab import files
from datasets import Dataset
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments
)

# Upload the lyrics text file
uploaded_files = files.upload()
input_file = list(uploaded_files.keys())[0]

# Read the file and clean lines
with open(input_file, "r", encoding="utf-8") as f:
    text_lines = [l.strip() for l in f if l.strip()]

# Create Hugging Face Dataset
lyrics_data = Dataset.from_dict({"text": text_lines})

# Load tokenizer and model
base_model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to EOS

model = GPT2LMHeadModel.from_pretrained(base_model_name)

# Tokenization function
def process(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
processed_dataset = lyrics_data.map(process, batched=True)

# Data collator for language modeling
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=200,
    logging_steps=20,
    overwrite_output_dir=True,
    save_total_limit=1,
    prediction_loss_only=True,
    logging_dir="./logs"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=tokenizer,
    data_collator=collator
)

# Start training
trainer.train()

# Save model and tokenizer to the specified folder
model.save_pretrained("gpt2-lyrics-finetuned")
tokenizer.save_pretrained("gpt2-lyrics-finetuned")

# Load fine-tuned model for generation
from transformers import pipeline

text_gen = pipeline("text-generation", model="gpt2-lyrics-finetuned", tokenizer="gpt2-lyrics-finetuned")

# Prompt for input
prompt_text = input("Enter your song prompt: ")

# Generate text
generated = text_gen(prompt_text, max_new_tokens=50)

# Display output
print("\n Generated Lyrics:")
print(generated[0]['generated_text'])


Saving paul-simon.txt to paul-simon.txt


Map:   0%|          | 0/2247 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-f0642a9bb4d8>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
20,4.305400
40,3.868800
60,4.157700
80,4.259300
100,3.804500
120,3.831500
140,4.011900
160,3.915100
180,3.646700
200,4.114800


Device set to use cuda:0


Enter your song prompt: I wanna follow 

 Generated Lyrics:
I wanna follow ____________ here, baby, here We boarded a train heading from New York to Los Angeles on the New York Southern Hemisphere highway, traveling along the watery Mississippi Delta for the Southern Hemisphere winter It was snowing in the winter of 1964, baby,
